**Importing Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Defining Paths**

In [2]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from google.colab.patches import cv2_imshow
import torchvision.transforms as transforms
from PIL import Image

def get_paths_of_an_object(root, object_name, folder="Val", type="Degraded_image" ):

 root=root+"/drive/MyDrive"

 dataset_path=os.path.join(root, "Denoising_Dataset_train_val")

 image_paths=[]



 for object in sorted(os.listdir(dataset_path)):

  if object==object_name:

    object_directory_path=os.path.join(root, "Denoising_Dataset_train_val", object)


    for train_folder in sorted(os.listdir(object_directory_path)):

      if(train_folder==folder): #validation folder

        train_directory_path=os.path.join(root, "Denoising_Dataset_train_val", object, train_folder)


        for degraded_image_folder in os.listdir(train_directory_path):

            if(degraded_image_folder==type):



                image_folder_path=os.listdir(os.path.join(root, "Denoising_Dataset_train_val", object, train_folder, degraded_image_folder))

                for image_folder in sorted(image_folder_path):

                    image_path=os.listdir(os.path.join(root, "Denoising_Dataset_train_val", object, train_folder, degraded_image_folder, image_folder))

                    for image in sorted(image_path):

                        image_paths.append(os.path.join(root, "Denoising_Dataset_train_val", object, train_folder, degraded_image_folder, image_folder, image))

 return image_paths

In [13]:
import os
import shutil

# Specify the path to the denoised_images folder
denoised_images_path = '/content/denoised_images/'

# Check if the directory exists
if os.path.exists(denoised_images_path):
    # Remove all contents of the directory
    shutil.rmtree(denoised_images_path)
    # Recreate the empty denoised_images folder
    os.makedirs(denoised_images_path)
    print("All contents of the denoised_images folder have been deleted.")
else:
    print("The specified directory does not exist.")


All contents of the denoised_images folder have been deleted.


**Model Architecture**

In [5]:
import torch
import torch.nn as nn
import cv2
import numpy as np
from torchvision import transforms

# Step 1: Define your model architecture
import torch.nn.init as init
device = "cuda" if torch.cuda.is_available() else "cpu"

import torch.nn.init as init

class DnCNN(nn.Module):

    def __init__(self, depth=15, n_channels=64, image_channels=1, use_bnorm=True, kernel_size=3):

        super(DnCNN, self).__init__()

        kernel_size = 3

        padding = 1

        layers = []



        layers.append(nn.Conv2d(in_channels=image_channels, out_channels=n_channels, kernel_size=kernel_size, padding=padding, bias=True))

        layers.append(nn.ReLU(inplace=True))

        for _ in range(depth-2):

            layers.append(nn.Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size, padding=padding, bias=False))

            layers.append(nn.BatchNorm2d(n_channels, eps=0.0001, momentum = 0.95))

            layers.append(nn.ReLU(inplace=True))

        layers.append(nn.Conv2d(in_channels=n_channels, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=False))

        self.dncnn = nn.Sequential(*layers)

        self._initialize_weights()



    def forward(self, x):

        y = x

        out = self.dncnn(x)

        denoised_output = y - out

        return torch.clamp(denoised_output, 0, 1)



    def _initialize_weights(self):

        for m in self.modules():

            if isinstance(m, nn.Conv2d):

                init.orthogonal_(m.weight)

                if m.bias is not None:

                    init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm2d):

                init.constant_(m.weight, 1)

                init.constant_(m.bias, 0)

# Step 2: Create a model instance
model = DnCNN(depth=14, n_channels=64, image_channels=3).to(device)

**Loading best model**

In [7]:


# Step 3: Load the state_dict with weights_only=True
model.load_state_dict(torch.load('/content/Best_model.pth'))
model.eval()  # Set the model to evaluation mode



<ipython-input-7-bb8a184c04a8>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/Best_model.pth'))


DnCNN(
  (dncnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (6): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(64, eps=0.0001, momentum=0.95, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): Conv2d(64, 64, kernel_size=(3,

**Objects Names**

In [8]:
def get_object_names(root): #returns paths of all the images of a given object

 root=root+"/drive/MyDrive"

 dataset_path=os.path.join(root, "Denoising_Dataset_train_val")

 object_names=[]



 for object in sorted(os.listdir(dataset_path)):

  object_names.append(object)



 return object_names



objects=get_object_names(os.getcwd())

objects

['bottle',
 'cable',
 'capsule',
 'carpet',
 'grid',
 'hazelnut',
 'leather',
 'metal_nut',
 'pill',
 'screw',
 'tile',
 'toothbrush',
 'transistor',
 'wood',
 'zipper']

In [9]:
import os

# Check if the main dataset directory exists
root = '/content/drive/MyDrive'  # or adjust to your actual root if needed
dataset_path = os.path.join(root, "Denoising_Dataset_train_val")
print("Checking dataset path:", dataset_path)

if not os.path.exists(dataset_path):
    print(f"Dataset path does not exist: {dataset_path}")
else:
    print("Dataset path exists. Contents:", os.listdir(dataset_path))


Checking dataset path: /content/drive/MyDrive/Denoising_Dataset_train_val
Dataset path exists. Contents: ['screw', 'transistor', 'hazelnut', 'toothbrush', 'tile', 'pill', 'zipper', 'metal_nut', 'wood', 'leather', 'grid', 'cable', 'bottle', 'capsule', 'carpet']


**Finding PSNR ANd saving denoised iamges in corresponding folders**

In [16]:
import os
import torch
from torchvision import transforms
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define your image transformations here
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.ToTensor(),            # Convert images to tensor
])

# Specify the main output directory for denoised images
output_directory = '/content/denoised_images'
os.makedirs(output_directory, exist_ok=True)  # Create the main directory if it doesn't exist

model.eval()

with torch.inference_mode():
    psnr_scores = []  # Initialize list to store PSNR scores

    for object in objects:
        degraded_object_image_paths = get_paths_of_an_object('/content', object)
        clean_object_image_paths = get_paths_of_an_object('/content', object, "Val", "GT_clean_image")

        # Load and transform degraded images
        degraded_images = torch.zeros(len(degraded_object_image_paths), 3, 256, 256)
        for i, noisy_image in enumerate(degraded_object_image_paths):
            pil_image = Image.open(noisy_image)
            noisy = transform(pil_image)  # Use the defined transform
            degraded_images[i] = noisy

        # Load and transform clean images
        clean_images = torch.zeros(len(clean_object_image_paths), 3, 256, 256)
        for i, image in enumerate(clean_object_image_paths):
            pil_image = Image.open(image)
            image = transform(pil_image)  # Use the defined transform
            clean_images[i] = image

        # Pass degraded images through the model
        reconstructed_images = model(degraded_images.to(device))

        # Convert tensors to numpy arrays for PSNR calculation
        reconstructed_images_np = reconstructed_images.cpu().detach().numpy()
        clean_images_np = clean_images.cpu().detach().numpy()

        # Calculate PSNR
        psnr_score = psnr(reconstructed_images_np, clean_images_np)  # Calculate PSNR
        psnr_scores.append(psnr_score)

        print(f"PSNR for {object} is {psnr_score:.2f}")

        # Create a subdirectory for the current object category
        category_directory = os.path.join(output_directory, object)
        os.makedirs(category_directory, exist_ok=True)  # Create the category directory if it doesn't exist

        # Save the denoised images in the category directory
        for i in range(reconstructed_images_np.shape[0]):
            # Convert the denoised tensor to a PIL image
            denoised_image = transforms.ToPILImage()(reconstructed_images[i].cpu())
            # Define the path to save the image
            denoised_image_path = os.path.join(category_directory, f"{object}_denoised_{i}.png")
            # Save the image
            denoised_image.save(denoised_image_path)


PSNR for bottle is 26.01
PSNR for cable is 23.96
PSNR for capsule is 24.55
PSNR for carpet is 21.17
PSNR for grid is 23.29
PSNR for hazelnut is 28.96
PSNR for leather is 27.58
PSNR for metal_nut is 26.81
PSNR for pill is 28.24
PSNR for screw is 24.99
PSNR for tile is 23.59
PSNR for toothbrush is 27.24
PSNR for transistor is 26.38
PSNR for wood is 26.32
PSNR for zipper is 26.89
